In [6]:

import matplotlib.pyplot as plt
import cv2
import numpy as np
import pandas as pd
from glob import glob
import re
from tqdm import tqdm_notebook as tqdm
from pathlib import Path
import os

from faceDetectorS3fd.face_detector.detect_engine import FaceDetectionEngine

detector = FaceDetectionEngine(weights_path="faceDetectorS3fd/face_detector/s3fd_convert.pth")



In [7]:
import fnmatch
import os
import re
from pathlib import Path


def resize_by_height(img, new_height):
    '''
    Input: image, new_height
    Returns: image with new height and scaled width
    '''
    height, width = img.shape[:2]
    resize_multiple = new_height / height
    img = cv2.resize(img, None, fx=resize_multiple, fy=resize_multiple, interpolation=cv2.INTER_AREA)
    return img


def get_face_crop(img):

    results = anfas_detector.detect_faces(img)
    if results == []: 
        return results, None

    x, y, w, h = results[0]['box']

    y_border, x_border = img.shape[:2]
    x1, y1, x2, y2 = max(0, x), max(0, y), min(x_border, x+w), min(y_border, y+h)

    face_crop = img[y1:y2, x1:x2]
    return face_crop, round(results[0]['confidence'], 2)


def get_all_fnames(base_folder):
    all_fnames = glob(str(Path(base_folder, '**', '*')), recursive=True)
    all_imgs = []
    patterns = ['*jpg', '*jpeg', '*png']

    for pattern in patterns:
        match = re.compile(fnmatch.translate(pattern), re.IGNORECASE).match
        valid_pths = [pth for pth in all_fnames if match(pth)]
        all_imgs.extend(valid_pths)
                      
    return all_imgs

In [15]:
# -*- coding: utf-8 -*-
import cv2
import numpy




base_folder = '/home/nikolay/Documents/MUST/DS Drivers Faces-20200704T025843Z-001/DS Drivers Faces/Копии документов'
result_base_folder = Path(f'{base_folder}_result')
if not os.path.exists(result_base_folder):
    os.mkdir(result_base_folder)
    
    
all_imgs = get_all_fnames(base_folder)
for num, img_path in enumerate(tqdm(all_imgs[:])):
    folder_name = Path(img_path).parts[-2]
    img_name = Path(img_path).parts[-1]
    
#     stream = open(f'{img_path}', "rb")
#     bytes_ = bytearray(stream.read())
#     numpyarray = np.asarray(bytes_, dtype=numpy.uint8)
#     img = cv2.imdecode(numpyarray, cv2.IMREAD_UNCHANGED)

    img = cv2.imread(img_path)
#     print(img.shape, img_path)

    img = resize_by_height(img, 512)
    
#     plt.imshow(img)
#     plt.show()
    preds = detector.predict(img, dilate_bbox=True)
    if len(preds) > 0: crop = detector.crop(img, preds[0])

    if len(preds) == 0: 
        img = cv2.rotate(img, rotateCode = 0)
        preds = detector.predict(img, dilate_bbox=True)
        if len(preds) > 0: crop = detector.crop(img, preds[0])

        if len(preds) == 0: 
            img = cv2.rotate(img, rotateCode = 0)
            preds = detector.predict(img, dilate_bbox=True)
            if len(preds) > 0: crop = detector.crop(img, preds[0])

            if len(preds) == 0: 
                img = cv2.rotate(img, rotateCode = 0)
                preds = detector.predict(img, dilate_bbox=True)
                if len(preds) > 0: crop = detector.crop(img, preds[0])
                continue
                
        
    if not os.path.exists(result_base_folder/folder_name):
        os.mkdir(result_base_folder/folder_name)
        
    cv2.imwrite(str(result_base_folder/folder_name/Path(img_name)), crop)
    
    
    if num % 100 == 0:
        with open('progress.txt', 'w') as f:
            f.write(str(num))
            
#     plt.imshow(crop)
#     plt.show()

/home/nikolay/.pyenv/versions/3.7.3/envs/main/lib/python3.7/site-packages/ipykernel_launcher.py:15: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  from ipykernel import kernelapp as app


# Check what folders were not detected with faces

In [21]:
base_folder = '/home/nikolay/Documents/MUST/DS Drivers Faces-20200704T025843Z-001/DS Drivers Faces/Копии документов'
base_folder_names = pd.Series(os.listdir(base_folder))
base_folder_names

0      434
1      040
2      111
3      047
4      242
      ... 
470    194
471    200
472    291
473    163
474    294
Length: 475, dtype: object

In [22]:
result_folder = base_folder+'_result'
result_folder_names = pd.Series(os.listdir(result_folder))
result_folder_names

0      040
1      111
2      047
3      242
4      339
      ... 
450    194
451    200
452    291
453    163
454    294
Length: 455, dtype: object

In [27]:
base_folder_names[~base_folder_names.isin(result_folder_names)]

0          434
9          432
24         360
55         376
91         347
130        014
142        471
168        361
202        472
209    454 (1)
242        422
248        346
270        173
304         72
355        462
365        401
370        275
371    145 (1)
413        431
461        414
dtype: object